# NER Code to return medical entities from Unstructured data

In [ ]:
#installing required libraries
!pip install -q -U spacy
!pip install -q scispacy
!pip install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz

In [ ]:
# @title Function 'fetchEntity' to accept text and return the list of entities extracted from the text
import scispacy
import spacy
import en_core_sci_sm
def fetchEntity(unstructuredText="no input"):
  if unstructuredText=="no input":
    print("No text was passed as input. You need to pass patient related text during function call.")
  else:
    nlp_sm = en_core_sci_sm.load()
    doc = nlp_sm(unstructuredText)
    entityList=list(doc.ents)
    #print("entityList is as follows: \n", entityList)
    return entityList

# Codisep_Knowledge_Graph

In [ ]:
!pip install word2number

In [ ]:
from word2number import w2n
import inflect

In [ ]:
def remove_duplicates_and_empty(lst):
    return list(set(filter(None, lst)))

# Example usage
input_list = ["apple", "banana", "apple", "", "orange", "banana", ""]
result = remove_duplicates_and_empty(input_list)
print(result)  # Output: ["apple", "banana", "orange"]


In [ ]:
import re

def remove_special_characters_and_digits(input_string):
    # Use regular expression to remove non-space characters that are not letters
    cleaned_string = re.sub(r'[^a-zA-Z\s]', '', input_string)
    return cleaned_string

# Example usage
input_string = "Hello! 123 This is a test string with special characters."
cleaned_result = remove_special_characters_and_digits(input_string)
print(cleaned_result)  # Output: "Hello  This is a test string with special characters"


In [ ]:
import re

def keep_only_english_characters(input_string):
    # Remove numbers and special characters
    cleaned_string = re.sub(r'[^a-zA-Z]', '', input_string)

    # Remove non-English characters
    cleaned_string = re.sub(r'[^\x00-\x7F]', '', cleaned_string)

    return cleaned_string.lower()

In [ ]:
import csv

# Specify the path to the TSV file
tsv_file_path = '/content/codisep_data_sorted_final.tsv'


#Getting required items
patient_ids=[]
medical_codes=[]
medical_codes_description=[]
medical_concepts=[]
medical_synonyms=[]
medical_definitions=[]


# Open the TSV file using a context manager
with open(tsv_file_path, 'r', newline='', encoding='utf-8') as tsv_file:
    # Create a CSV reader object with tab as the delimiter
    tsv_reader = csv.reader(tsv_file, delimiter='\t')

    # Loop through the rows in the TSV file
    for row in tsv_reader:
        # Each row is a list of values, where each value corresponds to a column
        # Process the values in the row as needed
        #print(row[2])
        patient_ids.append(row[0])
        medical_codes.append(row[1])
        medical_codes_description.append(row[2].lower())
        medical_concepts.append(row[3].lower())
        input_string = row[4]
        split_parts = input_string.split("@@@@@")
        #print(split_parts)  # Output: ['username', 'example.com']
        for i in split_parts:
          medical_definitions.append(i.lower())
        input_string = row[5]
        split_parts = input_string.split("@@@@@")
        for i in split_parts:
          medical_synonyms.append(i.lower())



In [ ]:
patient_ids=remove_duplicates_and_empty(patient_ids)
medical_codes=remove_duplicates_and_empty(medical_codes)
medical_codes_description=remove_duplicates_and_empty(medical_codes_description)
medical_concepts=remove_duplicates_and_empty(medical_concepts)
medical_definitions=remove_duplicates_and_empty(medical_definitions)
medical_synonyms=remove_duplicates_and_empty(medical_synonyms)

cypher_query_codisep=[]
max_lenght=[]
max_lenght.append(patient_ids)
max_lenght.append(medical_codes)
max_lenght.append(medical_codes_description)
max_lenght.append(medical_concepts)
max_lenght.append(medical_definitions)
max_lenght.append(medical_synonyms)




len(medical_synonyms)

# Creating the Knowledge Graph Nodes

Patient Nodes

In [ ]:
n=0
for i in patient_ids:
  n=n+1
  p = inflect.engine()
  numeric_value = n
  word = p.number_to_words(numeric_value)
  cypher_query_codisep.append("("+i+":patient { name: "+'"'+"Patient "+word+'"'+" }),")

Medical Code Nodes

In [ ]:
for i in medical_codes:
  cypher_query_codisep.append("("+i.lower()+":medical_code { name: "+'"'+i+'"'+" }),")

Medical Code Descriptions

In [ ]:
for i in medical_codes_description:
  cypher_query_codisep.append("("+keep_only_english_characters(i.lower())+":medical_code_description { name: "+'"'+i+'"'+" }),")

Medical Concept Nodes

In [ ]:
for i in medical_concepts:
  cypher_query_codisep.append("("+keep_only_english_characters(i.lower())+":medical_concept { name: "+'"'+i+'"'+" }),")

Medical Synonyms

In [ ]:
for i in medical_synonyms:
  cypher_query_codisep.append("("+keep_only_english_characters(i.lower())+":medical_synonym { name: "+'"'+i+'"'+" }),")

Medical Definitions

In [ ]:
for i in medical_definitions:
  cypher_query_codisep.append("("+keep_only_english_characters(i.lower())+":medical_definition { name: "+'"'+i+'"'+" }),")

In [ ]:
cypher_query_codisep

In [ ]:
for line in cypher_query_codisep:
  with open("cypher_query.txt", "a") as query:
    query.write(line)

# Taking Data from user. (Structured Data)

In [ ]:
def take_data():
 concept = input("Add desired medical concept: ")
 check = int(input("Press 1 to add another and 0 to abort: "))  # Convert input to integer
 user_defined_concepts.append(concept)
 try:
  if check==1:
   take_data()
  else:
   pass
 except:
   pass

In [ ]:
# Get user data at runtime with type conversion
user_defined_concepts=[]
take_data()

user_defined_unique_concepts = list(set(user_defined_concepts))

# Display the collected data
print("User Entered Concepts:", user_defined_unique_concepts)



# Bioportal Section

In [ ]:
import urllib.request, urllib.error, urllib.parse
import json
import os
from pprint import pprint
REST_URL = "http://data.bioontology.org"
API_KEY = "396993d0-4ce2-4123-93de-214e9b9ebcf2"

In [ ]:
def get_json(url):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
    return json.loads(opener.open(url).read())

# Functions For Knowledge Filteration
1. Mainly used for Knowledge Graph Nodes Key Generation

In [ ]:
import re

def keep_only_english_characters(input_string):
    # Remove numbers and special characters
    cleaned_string = re.sub(r'[^a-zA-Z]', '', input_string)

    # Remove non-English characters
    cleaned_string = re.sub(r'[^\x00-\x7F]', '', cleaned_string)

    return cleaned_string.lower()

def contains_english_character(input_string):
    for char in input_string:
        ascii_value = ord(char)
        if 65 <= ascii_value <= 122:
            return True
    return False

def remove_quotes(input_string):
    cleaned_string = input_string.replace('"', '').replace("'", '')
    return cleaned_string

# Example usage
original_string = 'This is a "quoted" string with some \'quotes\'.'
cleaned_string = remove_quotes(original_string)
print(cleaned_string)


In [ ]:
keep_only_english_characters("fig 18 abbr: ta-i1,ta-i4,etc. in isbn:0-937548-00-6.")

# Fuzzy Matching to remove duplicates

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#defining threshold
similarity_threshold = 80

def remove_duplicates(strings):
    duplicates = set()
    unique_strings = []

    for string1 in strings:
        is_duplicate = False
        for string2 in unique_strings:
            similarity_score = fuzz.token_sort_ratio(string1, string2)
            if similarity_score >= similarity_threshold:
                is_duplicate = True
                duplicates.add(string2)
                break
        if not is_duplicate:
            unique_strings.append(string1)

    return unique_strings, duplicates


In [ ]:
sentences = [
    "This is a sample sentence.",
    "This is a sample sentence",
    "Another example sentence.",
    "Yet another example.",
    "This is a similar sentence.",
]

unique_sentences, duplicates = remove_duplicates(sentences)

print("Unique Sentences:")
for sentence in unique_sentences:
    print(sentence)

print("\nDuplicates:")
for duplicate in duplicates:
    print(duplicate)


# Bioportal Synonyms and Definitions retrieval function

In [ ]:
def get_synonyms(concept):
  search_results,synonym,definition = [],[],[]
  syno_fin=""
  search_results.append(get_json(REST_URL + "/search?q=" + concept)["collection"])
  #collecting enriched knowledge and adding that in respective lists.
  for result in search_results:
    for i in range(len(result)):
     data=result[i]
     for k in dict.keys(data):
       if k=="synonym":
         for i in data["synonym"]:
           synonym.append(i)
       if k=="definition":
         for i in data["definition"]:
           definition.append(i)

  #removing duplicates synonyms and creating unique list
  syn_new=[]

  for i in synonym:
   syn_new.append(i.lower())

  myset = set(syn_new)
  syno_new=list(myset)
  #print(syno_new)
  syno_new = list(dict.fromkeys(syno_new))
  unique_synonyms, duplicates = remove_duplicates(syno_new)
  #adding definition nodes and their relationships in graphs
  for i in unique_synonyms:
    if contains_english_character(i)==True:
     print(i)
     id=keep_only_english_characters(i)
     syno_fin=syno_fin+i+"@@@@@"
    else:
      pass
  return syno_fin


In [ ]:
def get_definitions(concept):
  search_results,synonym,definition = [],[],[]
  def_fin=""
  search_results.append(get_json(REST_URL + "/search?q=" + concept)["collection"])
  #collecting enriched knowledge and adding that in respective lists.
  for result in search_results:
    for i in range(len(result)):
     data=result[i]
     for k in dict.keys(data):
       if k=="synonym":
         for i in data["synonym"]:
           synonym.append(i)
       if k=="definition":
         for i in data["definition"]:
           definition.append(i)

  #removing duplicates definitions and creating unique list
  definintion_unique,new_define=[],[]
  for i in definition:
   new_define.append(i.lower())

  defset=set(new_define)
  definintion_unique=list(defset)
  #print(definintion_unique)

  #adding definition nodes and their relationships in graphs
  if definintion_unique!="":
   definintion_unique = list(dict.fromkeys(definintion_unique))

  unique_sentences, duplicates = remove_duplicates(definintion_unique)
  for i in unique_sentences:
   if contains_english_character(i)==True:
    id=keep_only_english_characters(i)
    def_fin=def_fin+i+"@@@@@"
   else:
    pass
  return def_fin



# Knowledge Graph or Cypher Query Creation

In [ ]:
from urllib.parse import quote

data_sorted=[]
data_sorted_file_insertion=[]

for i in user_defined_unique_concepts:
 data_sorted.append(i)
 encoded_query = quote(i)
 data_sorted.append(get_definitions(encoded_query))
 data_sorted.append(get_synonyms(encoded_query))
 data_sorted_file_insertion.append(data_sorted)
 data_sorted=[]

print(data_sorted_file_insertion)

# TSV file used as a backup/ for future purpose

In [ ]:
import csv


# Path for the new TSV file
tsv_file_path = 'graph_data_sorted.tsv'

# Write data to the TSV file
with open(tsv_file_path, 'w', newline='', encoding='utf-8') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    writer.writerows(data_sorted_file_insertion)

print(f"New TSV file '{tsv_file_path}' has been created.")

In [ ]:
cypher_query=[]
cypher_query.append("create ")
f= open("cypher_query.txt","w+")
f.close()
with open("cypher_query.txt", "a") as query:
  query.write("Create")

In [ ]:

for i in data_sorted_file_insertion:
  x=0
  for n in i:
    if x==0:
      cypher_query.append("("+n.lower()+"concept:medical_concept { name: "+'"'+remove_quotes(n)+'"'+" }),")
    # for medical definitions
    elif x==1:
      # Split the string using @@@@@ as the delimiter
      split_list = n.split("@@@@@")
      for m in split_list:
        cypher_query.append("("+keep_only_english_characters(m.lower())+":definition { name: "+'"'+remove_quotes(m)+'"'+" }),")
        cypher_query.append("("+keep_only_english_characters(m.lower())+")-[:Definition]->("+i[0].lower()+"concept),")

    # for medical synonyms
    elif x==2:
      # Split the string using @@@@@ as the delimiter
      split_list = n.split("@@@@@")
      for m in split_list:
        cypher_query.append("("+keep_only_english_characters(m.lower())+":synonyms { name: "+'"'+remove_quotes(m)+'"'+" }),")
        cypher_query.append("("+keep_only_english_characters(m.lower())+")-[:Synonym]->("+i[0].lower()+"concept),")
    x=x+1


In [ ]:
# @title Remove ending comma from the last string within the list cypher_query
#remove last comma from the ending string from within the list cypher_query
print(cypher_query[len(cypher_query)-1])
lastString=cypher_query[len(cypher_query)-1]
length=len(lastString)
trimmedLastString=lastString[:length-1]
print(trimmedLastString)
cypher_query[len(cypher_query)-1]=trimmedLastString

# Text File creation

In [ ]:

# Specify the file path
file_path = 'cypher_query.txt'

# Open the file for writing
with open(file_path, 'w') as file:
    # Write each item to a separate line in the file
    for item in cypher_query:
        file.write(item)

print(f"Items written to {file_path}")


# README



------------------------------------------------------
# New Section
1. Now need to work on duplicate nodes and have to create relationships-->(Done--99% automated, 1% chance of erros incase if generic terms are used to create KG)--> Can add instructions.

2. For next week, need to create connections among concepts (use Neo4j algorithms or use word embeddings)